# Examinations Graph

## Import Moduli

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, arrays_zip, monotonically_increasing_id, lit, split, to_date, upper, when
from pyspark.ml import Pipeline

print("Apache Spark version: ", spark.version)

spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

Apache Spark version: 3.1.0

## Neo4j Config

In [ ]:
# Sostituire myip con l'IP su cui è installato il db
url = "bolt://myip:7687"
username = "neo4j"
password = "neo4j"

## Lettura Dataset

### Visite

In [ ]:
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","
path = "/FileStore/tables/DF_VISITA_COMPLETO.csv"


exams = spark.read.format(file_type) \
             .option("inferSchema", infer_schema) \
             .option("header", first_row_is_header) \
             .option("multiLine", "true") \
             .option("sep", delimiter) \
             .load(path)

exams = exams.withColumn("DATA_EVENTO", to_date(col("DATA_EVENTO"), "yyyy-MM-dd")) \
             .replace("null", None) \
             .withColumn("ESAMINATORE", upper(col("ESAMINATORE")))

### Anamnesi

In [ ]:
path = "/FileStore/tables/anamnesi_completa.json"

res_anamnesi = spark.read.format("json") \
                    .option("inferSchema", "true") \
                    .load(path)

In [ ]:
print("Anamnesi analizzate dal NER:", res_anamnesi.count())

Anamnesi analizzate dal NER: 2012

In [ ]:
entities_anamnesi = res_anamnesi.select("ID_VISITA", "chunk.result", "chunk.metadata") \
                                .withColumn("tmp", arrays_zip("result", "metadata")) \
                                .withColumn("tmp", explode("tmp")) \
                                .select("ID_VISITA", "tmp.result", "tmp.metadata.entity") \
                                .withColumnRenamed("result", "chunk") \
                                .withColumn("chunk", upper(col("chunk")))

In [ ]:
print("Coppie (visita, entity) di anamnesi:", entities_anamnesi.count())
print("Coppie (visita, entity) di anamnesi distinte:", entities_anamnesi.distinct().count())

Coppie (visita, entity) di anamnesi: 7400
Coppie (visita, entity) di anamnesi distinte: 7348

In [ ]:
entities_anamnesi = entities_anamnesi.drop_duplicates()

print("Coppie (visita, entity) di anamnesi:", entities_anamnesi.count())

Coppie (visita, entity) di anamnesi: 7348

### Diagnosi

In [ ]:
path = "/FileStore/tables/diagnosi_completa.json"

res_diagnosi = spark.read.format("json") \
                    .option("inferSchema", "true") \
                    .load(path)

In [ ]:
print("Diagnosi analizzate dal NER:", res_diagnosi.count())

Diagnosi analizzate dal NER: 134094

In [ ]:
entities_diagnosi = res_diagnosi.select("ID_VISITA", "chunk.result", "chunk.metadata") \
                                .withColumn("tmp", arrays_zip("result", "metadata")) \
                                .withColumn("tmp", explode("tmp")) \
                                .select("ID_VISITA", "tmp.result", "tmp.metadata.entity") \
                                .withColumnRenamed("result", "chunk") \
                                .withColumn("chunk", upper(col("chunk")))

In [ ]:
print("Coppie (visita, entity) di diagnosi:", entities_diagnosi.count())
print("Coppie (visita, entity) di diagnosi distinte:", entities_diagnosi.distinct().count())

Coppie (visita, entity) di diagnosi: 412122
Coppie (visita, entity) di diagnosi distinte: 411020

In [ ]:
entities_diagnosi = entities_diagnosi.drop_duplicates()

print("Tuple (paziente, medico, data, entità):", entities_diagnosi.count())

Tuple (paziente, medico, data, entità): 411020

### Farmaci

In [ ]:
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","
path = "/FileStore/tables/DF_FARMACI_COMPLETO.csv"

drugs = spark.read.format(file_type) \
             .option("inferSchema", infer_schema) \
             .option("header", first_row_is_header) \
             .option("multiLine", "true") \
             .option("sep", delimiter) \
             .load(path)

drugs = drugs.replace("null", None).dropna(subset=["FARMACO_FINALE"])

In [ ]:
display(drugs.limit(5))

FARMACO_FINALE,ID_VISITA,CONFEZIONE,DOSE
SIRDALUD,34,SIRDALUD,un/una al mattino e sera
NORVASC,143,NORVASC 5,1 cp al pomeriggio
ENAPREN,207,ENAPREN 20,un/una al mattino
TOTALIP,242,TOTALIP 20,un/una alla sera
ISOPTIN,324,ISOPTIN 240,un/una al mattino


## Entità

### Pazienti

In [ ]:
patients = exams.select("CODPAZ") \
                .distinct() \
                .withColumnRenamed("CODPAZ", "codice") \
                .orderBy("codice")


print("Pazienti distinti:", patients.count())

Pazienti distinti: 1363

In [ ]:
patients.write.format("org.neo4j.spark.DataSource") \
        .option("url", url) \
        .option("authentication.type", "basic") \
        .option("authentication.basic.username", username) \
        .option("authentication.basic.password", password) \
        .option("labels", ":Paziente") \
        .option("node.keys", "codice") \
        .mode("Overwrite") \
        .save()

### Dottori

In [ ]:
doctors = exams.select("COD_MEDICO_FIRMANTE", "ESAMINATORE") \
               .distinct() \
               .withColumnRenamed("COD_MEDICO_FIRMANTE", "codice") \
               .withColumnRenamed("ESAMINATORE", "nome") \
               .orderBy("codice") \
               .dropna(subset=["codice"])

print("Medici distinti:", doctors.count())

Medici distinti: 17

In [ ]:
display(doctors.distinct())

codice,nome
1_11,CARMINE MORISCO
1_13,M.A.E. RAO
1_15,null
1_16_1,RAFFAELE IZZO
1_16_3,N. DE LUCA
1_19,null
1_20,null
1_21,ORESTE ARCUCCI
1_22,null
1_22_1,LETIZIA SPINELLI


In [ ]:
doctors.write.format("org.neo4j.spark.DataSource") \
        .option("url", url) \
        .option("authentication.type", "basic") \
        .option("authentication.basic.username", username) \
        .option("authentication.basic.password", password) \
        .option("labels", ":Medico") \
        .option("node.keys", "codice") \
        .option("node.properties", "nome") \
        .mode("Overwrite") \
        .save()

### Visita

In [ ]:
exams_ = exams.select("ID_VISITA", "DATA_EVENTO") \
              .distinct() \
              .withColumnRenamed("ID_VISITA", "id") \
              .withColumnRenamed("DATA_EVENTO", "data") \
              .orderBy("id")

print("Visite distinte:", exams_.count())

Visite distinte: 147599

In [ ]:
display(exams_.limit(5))

id,data
0,2008-03-27
1,2008-06-10
2,2019-03-29
3,2019-05-03
4,2019-11-25


In [ ]:
exams_.write.format("org.neo4j.spark.DataSource") \
      .option("url", url) \
      .option("authentication.type", "basic") \
      .option("authentication.basic.username", username) \
      .option("authentication.basic.password", password) \
      .option("labels", ":Visita") \
      .option("node.keys", "id, data") \
      .mode("Overwrite") \
      .save()

### Malattia

In [ ]:
disease_anamnesi =  entities_anamnesi.filter("entity == 'Disease'") \
                                     .select("chunk") \
                                     .withColumnRenamed("chunk", "nome") \
                                     .distinct()

disease_diagnosi =  entities_diagnosi.filter("entity == 'Disease'") \
                                     .select("chunk") \
                                     .withColumnRenamed("chunk", "nome") \
                                     .distinct()

print("Malattie di anamnesi distinte:", disease_anamnesi.count())
print("Malattie di diagnosi distinte:", disease_diagnosi.count())

Malattie di anamnesi distinte: 1183
Malattie di diagnosi distinte: 17061

In [ ]:
diseases = disease_anamnesi.union(disease_diagnosi).distinct()

print("Malattie distinte:", diseases.count())

Malattie distinte: 17462

In [ ]:
diseases.write.format("org.neo4j.spark.DataSource") \
        .option("url", url) \
        .option("authentication.type", "basic") \
        .option("authentication.basic.username", username) \
        .option("authentication.basic.password", password) \
        .option("labels", ":Malattia") \
        .option("node.keys", "nome") \
        .mode("Overwrite") \
        .save()

### Sintomo

In [ ]:
symptom_anamnesi =  entities_anamnesi.filter("entity == 'Symptom'") \
                                     .select("chunk") \
                                     .withColumnRenamed("chunk", "nome") \
                                     .distinct()

symptom_diagnosi =  entities_diagnosi.filter("entity == 'Symptom'") \
                                     .select("chunk") \
                                     .withColumnRenamed("chunk", "nome") \
                                     .distinct()

print("Sintomi di anamnesi distinti:", symptom_anamnesi.count())
print("Sintomi di diagnosi distinti:", symptom_diagnosi.count())

Sintomi di anamnesi distinti: 513
Sintomi di diagnosi distinti: 5955

In [ ]:
symptoms = symptom_anamnesi.union(symptom_diagnosi).distinct()

print("Sintomi distinti:", symptoms.count())

Sintomi distinti: 6238

In [ ]:
symptoms.write.format("org.neo4j.spark.DataSource") \
        .option("url", url) \
        .option("authentication.type", "basic") \
        .option("authentication.basic.username", username) \
        .option("authentication.basic.password", password) \
        .option("labels", ":Sintomo") \
        .option("node.keys", "nome") \
        .mode("Overwrite") \
        .save()

### Farmaco

In [ ]:
drugs_ = drugs.select("FARMACO_FINALE") \
              .distinct() \
              .withColumnRenamed("FARMACO_FINALE", "nome") \
              .orderBy("nome")

print("Farmaci distinti:", drugs_.count())

Farmaci distinti: 4303

In [ ]:
display(drugs_)

nome
A
A PARTIRE DA GG PRIMA
A PARTIRE DA SABATO
A TEM AEREOSOL DOSATO
ABASAGLAR KWIKPEN
ABATACEPT
ABBA
ABIETTA
ABILIFY
ABOLIRE FUMO DI SIGARETT


In [ ]:
drugs_.write.format("org.neo4j.spark.DataSource") \
      .option("url", url) \
      .option("authentication.type", "basic") \
      .option("authentication.basic.username", username) \
      .option("authentication.basic.password", password) \
      .option("labels", ":Farmaco") \
      .option("node.keys", "nome") \
      .mode("Overwrite") \
      .save()

## Relazioni

### Paziente - Visita

In [ ]:
patient_exam =  exams.select("ID_VISITA", "CODPAZ", "SESSO", "ETA", "PESO", "ALTEZZA") \
                    .distinct() \
                    .withColumnRenamed("ID_VISITA", "id") \
                    .withColumnRenamed("CODPAZ", "codice") \
                    .orderBy("id")

In [ ]:
print(patient_exam.count())

147599

In [ ]:
patient_exam.write.format("org.neo4j.spark.DataSource")\
            .option("url", url) \
            .option("authentication.type", "basic") \
            .option("authentication.basic.username", username) \
            .option("authentication.basic.password", password) \
            .option("relationship", "SI_SOTTOPONE") \
            .option("relationship.save.strategy", "keys") \
            .option("relationship.source.labels", ":Paziente") \
            .option("relationship.source.save.mode", "overwrite") \
            .option("relationship.source.node.keys", "codice") \
            .option("relationship.target.labels", ":Visita") \
            .option("relationship.target.node.keys", "id") \
            .option("relationship.target.save.mode", "overwrite") \
            .option("relationship.properties", "SESSO:sesso, ETA:eta, PESO:peso, ALTEZZA:altezza")\
            .mode("Overwrite") \
            .save()

### Dottore - Visita

In [ ]:
doctor_exam =  exams.select("ID_VISITA", "COD_MEDICO_FIRMANTE") \
                    .distinct() \
                    .withColumnRenamed("ID_VISITA", "id") \
                    .withColumnRenamed("COD_MEDICO_FIRMANTE", "codice") \
                    .orderBy("id") \
                    .dropna()

In [ ]:
print(doctor_exam.count())

80762

In [ ]:
doctor_exam.write.format("org.neo4j.spark.DataSource")\
            .option("url", url) \
            .option("authentication.type", "basic") \
            .option("authentication.basic.username", username) \
            .option("authentication.basic.password", password) \
            .option("relationship", "EFFETTUA") \
            .option("relationship.save.strategy", "keys") \
            .option("relationship.source.labels", ":Medico") \
            .option("relationship.source.save.mode", "overwrite") \
            .option("relationship.source.node.keys", "codice") \
            .option("relationship.target.labels", ":Visita") \
            .option("relationship.target.node.keys", "id") \
            .option("relationship.target.save.mode", "overwrite") \
            .mode("Overwrite") \
            .save()

### Malattia - Visita

#### Anamnesi

In [ ]:
disease_anamnesi = entities_anamnesi.filter("entity == 'Disease'") \
                                   .select("ID_VISITA", "chunk") \
                                   .withColumnRenamed("chunk", "nome") \
                                   .withColumnRenamed("ID_VISITA", "id")

In [ ]:
disease_anamnesi.write.format("org.neo4j.spark.DataSource")\
                .option("url", url) \
                .option("authentication.type", "basic") \
                .option("authentication.basic.username", username) \
                .option("authentication.basic.password", password) \
                .option("relationship", "ANAMNESI") \
                .option("relationship.save.strategy", "keys") \
                .option("relationship.source.labels", ":Malattia") \
                .option("relationship.source.save.mode", "overwrite") \
                .option("relationship.source.node.keys", "nome") \
                .option("relationship.target.labels", ":Visita") \
                .option("relationship.target.node.keys", "id") \
                .option("relationship.target.save.mode", "overwrite") \
                .mode("Overwrite") \
                .save()

#### Diagnosi

In [ ]:
disease_diagnosi = entities_diagnosi.filter("entity == 'Disease'") \
                                   .select("ID_VISITA", "chunk") \
                                   .withColumnRenamed("chunk", "nome") \
                                   .withColumnRenamed("ID_VISITA", "id")

In [ ]:
print(disease_diagnosi.count())
print(disease_diagnosi.distinct().count())

370581
370581

In [ ]:
disease_diagnosi.write.format("org.neo4j.spark.DataSource")\
                .option("url", url) \
                .option("authentication.type", "basic") \
                .option("authentication.basic.username", username) \
                .option("authentication.basic.password", password) \
                .option("relationship", "DIAGNOSI") \
                .option("relationship.save.strategy", "keys") \
                .option("relationship.source.labels", ":Visita") \
                .option("relationship.source.save.mode", "overwrite") \
                .option("relationship.source.node.keys", "id") \
                .option("relationship.target.labels", ":Malattia") \
                .option("relationship.target.node.keys", "nome") \
                .option("relationship.target.save.mode", "overwrite") \
                .option("batch.size", "10000") \
                .mode("Overwrite") \
                .save()

### Sintomo - Visita

#### Anamnesi

In [ ]:
symptom_anamnesi = entities_anamnesi.filter("entity == 'Symptom'") \
                                   .select("ID_VISITA", "chunk") \
                                   .withColumnRenamed("chunk", "nome") \
                                   .withColumnRenamed("ID_VISITA", "id")

In [ ]:
print(symptom_anamnesi.count())
print(symptom_anamnesi.distinct().count())

1826
1826

In [ ]:
symptom_anamnesi.write.format("org.neo4j.spark.DataSource")\
                .option("url", url) \
                .option("authentication.type", "basic") \
                .option("authentication.basic.username", username) \
                .option("authentication.basic.password", password) \
                .option("relationship", "ANAMNESI") \
                .option("relationship.save.strategy", "keys") \
                .option("relationship.source.labels", ":Sintomo") \
                .option("relationship.source.save.mode", "overwrite") \
                .option("relationship.source.node.keys", "nome") \
                .option("relationship.target.labels", ":Visita") \
                .option("relationship.target.node.keys", "id") \
                .option("relationship.target.save.mode", "overwrite") \
                .option("batch.size", "10000") \
                .mode("Overwrite") \
                .save()

#### Diagnosi

In [ ]:
symptom_diagnosi = entities_diagnosi.filter("entity == 'Symptom'") \
                                   .select("ID_VISITA", "chunk") \
                                   .withColumnRenamed("chunk", "nome") \
                                   .withColumnRenamed("ID_VISITA", "id")

In [ ]:
print(symptom_diagnosi.count())
print(symptom_diagnosi.distinct().count())

40439
40439

In [ ]:
symptom_diagnosi.write.format("org.neo4j.spark.DataSource")\
                .option("url", url) \
                .option("authentication.type", "basic") \
                .option("authentication.basic.username", username) \
                .option("authentication.basic.password", password) \
                .option("relationship", "DIAGNOSI") \
                .option("relationship.save.strategy", "keys") \
                .option("relationship.source.labels", ":Visita") \
                .option("relationship.source.save.mode", "overwrite") \
                .option("relationship.source.node.keys", "id") \
                .option("relationship.target.labels", ":Sintomo") \
                .option("relationship.target.node.keys", "nome") \
                .option("relationship.target.save.mode", "overwrite") \
                .option("batch.size", "10000") \
                .mode("Overwrite") \
                .save()

### Farmaco - Visita

In [ ]:
print(drugs.dropna().count())
print(drugs.where(drugs.DOSE.isNull()).count())

547648
17051

In [ ]:
drugs.write.format("org.neo4j.spark.DataSource")\
      .option("url", url) \
      .option("authentication.type", "basic") \
      .option("authentication.basic.username", username) \
      .option("authentication.basic.password", password) \
      .option("relationship", "PRESCRITTO") \
      .option("relationship.save.strategy", "keys") \
      .option("relationship.source.labels", ":Visita") \
      .option("relationship.source.save.mode", "overwrite") \
      .option("relationship.source.node.keys", "ID_VISITA:id") \
      .option("relationship.target.labels", ":Farmaco") \
      .option("relationship.target.node.keys", "FARMACO_FINALE:nome") \
      .option("relationship.target.save.mode", "overwrite") \
      .option("relationship.properties", "CONFEZIONE:confezione, DOSE:dose")\
      .mode("Append") \
      .save()